In [3]:
import pandas as pd

In [29]:
import glob, os
import datetime

In [6]:
tickers = ["HDFCBANK", "ICICIBANK", "AXISBANK"]

In [7]:
source_data_path = "Data/StockPrice/{ticker}/"

## Combining individual stock part files

In [8]:
destination_data_path = "FINAL_DATA"

In [37]:
def combine_for_ticker(ticker):
    all_files = glob.glob(os.path.join(source_data_path.format(ticker=ticker), "*.csv"))
    all_dfs = {}
    for file in all_files:
        current_df = pd.read_csv(file)
        current_df["date"] = [datetime.datetime.strptime(x, "%d-%b-%Y").date() for x in current_df["Date"].values]
        current_df = current_df.drop(["Date", "Series", "Prev Close", "Last Price", "Average Price", "Turnover", "No. of Trades", "Deliverable Qty","% Dly Qt to Traded Qty"], axis=1)
        current_df = current_df.sort_values(by=['date'])
        close_price_changes = current_df["Close Price"] - current_df["Close Price"].shift()
        current_df["close_price_change"] = [1 if x > 0 else 0 for x in close_price_changes]
        all_dfs[file] = current_df
    total_df = pd.concat([all_dfs[x] for x in all_dfs], ignore_index=True)
    total_df.to_csv("{}/price_data_{}.csv".format(destination_data_path, ticker), header=True, index=False)

In [38]:
for ticker in tickers:
    combine_for_ticker(ticker)

## Combing all price data

In [39]:
files = ["FINAL_DATA/price_data_HDFCBANK.csv", "FINAL_DATA/price_data_ICICIBANK.csv", "FINAL_DATA/price_data_AXISBANK.csv"]

In [40]:
all_dfs = {}
for file in files:
    current_df = pd.read_csv(file)
    all_dfs[file] = current_df
total_df = pd.concat([all_dfs[x] for x in all_dfs], ignore_index=True)

In [41]:
total_df = total_df.sort_values(by=['date'])

In [44]:
total_df.head(n=10)

,Symbol,Open Price,High Price,Low Price,Close Price,Total Traded Quantity,date,close_price_change
1563,ICICIBANK,1100.90,1106.05,1092.00,1097.70,564158,2014-01-01,0
3125,AXISBANK,1301.50,1305.00,1288.20,1292.20,569885,2014-01-01,0
574,HDFCBANK,668.00,669.80,663.00,665.15,1123109,2014-01-01,0
3126,AXISBANK,1292.00,1325.00,1263.35,1272.30,2594412,2014-01-02,0
1564,ICICIBANK,1096.15,1118.00,1070.10,1075.60,2058457,2014-01-02,0
575,HDFCBANK,665.00,674.75,653.55,657.00,1807248,2014-01-02,0
1565,ICICIBANK,1068.00,1075.00,1045.00,1066.80,2830455,2014-01-03,0
3127,AXISBANK,1258.00,1269.50,1245.60,1260.90,2403748,2014-01-03,0
576,HDFCBANK,652.00,666.00,650.00,663.10,1628296,2014-01-03,1
3128,AXISBANK,1257.55,1262.00,1241.60,1256.30,1836289,2014-01-06,0


In [42]:
total_df.to_csv("FINAL_DATA/price_data.csv", header=True, index=False)